In [ ]:
import yt_api
from datetime import datetime, timedelta
import pandas as pd

In [ ]:
keyNum = 1
api =yt_api.youtubeAPI(keyNum)

In [ ]:
# Define the search query, max results, and start date
search_query = "Foreigner try filipino food"  # Your search query
max_results = 50  # Maximum results per query
start_date = datetime(2018, 8, 1)  # Start date
# Define the end date
end_date = datetime(2018, 8, 30)
# Create a time delta
delta = timedelta(days=10)

In [ ]:
all_videos = pd.DataFrame()
all_comments = pd.DataFrame()

In [ ]:
while start_date <= end_date:
    # Convert dates to the required format (ISO 8601)
    published_after = start_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    next_date = start_date + delta
    published_before = next_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    print(f"---Collecting data for {search_query} for Dates {published_after} to {published_before}....")
    # Call the search_vids function with the current date range
    try:
        response = api.get_vid_details_from_search(search_query, max_results, published_after, published_before)
    except Exception as e:
        print(f"Error in search: {e}... trying another api key")
        api = yt_api.youtubeAPI(keyNum+1)
        response = api.get_vid_details_from_search(search_query, max_results, published_after, published_before)

    temp = api.video_stats_to_dataframe(response)
    all_videos = pd.concat([all_videos,temp])
    vidIds = temp['videoId'].to_list()
    print(f"Collecting comments for: {vidIds}....")
    try:
        comment_list = api.get_comments_from_videos(vidIds)
    except Exception as e:
        print(f"Error here in get comments {e}..changing api key")
        api = yt_api.youtubeAPI(keyNum+1)
        comment_list = api.get_comments_from_videos(temp['videoId'].to_list())

    comments_df = pd.DataFrame(comment_list)
    all_comments = pd.concat([all_comments,comments_df])
    # Move to the next delta interval
    start_date = next_date

In [ ]:
all_videos.to_csv("videos_reacts_01-2018.csv",index=False)
all_comments.to_csv("comments_reacts_01-2018.csv",index=False)

## Filter Videos

In [ ]:
# Function to check if a channel is from the Philippines
def is_philippine_channel(channel_id):
    try:
        channel_info = api.channels().list(
            part='snippet',
            id=channel_id
        ).execute()
        
        country = channel_info['items'][0]['snippet']['country']
        
        return country == 'PH'  # 'PH' is the country code for the Philippines
    except Exception as e:
        print(f"Error checking channel {channel_id}: {str(e)}")
        return False

In [ ]:
def has_relevant_terms(video_id, relevant_terms):
    try:
        video_info = api.videos().list(
            part='snippet',
            id=video_id
        ).execute()
        
        snippet = video_info['items'][0]['snippet']
        
        # Check if the video title, description, or keywords contain relevant terms
        title = snippet.get('title', '').lower()
        description = snippet.get('description', '').lower()
        tags = [tag.lower() for tag in snippet.get('tags', [])]
        
        return any(term in title or term in description or term in tags for term in relevant_terms)
    except Exception as e:
        print(f"Error checking video {video_id}: {str(e)}")
        return False

In [ ]:
csv_file = ""

In [ ]:
df = pd.read_csv(csv_file)

# Define relevant terms related to the Philippines, including Tagalog words
relevant_terms = ['philippines', 'pinoy', 'tagalog', 'filipino', 'manila', 'filipina', 'pinas']

# Filter out channels that are not from the Philippines and videos with relevant terms
df['is_philippine'] = df['channelId'].apply(is_philippine_channel)
df['has_relevant_terms'] = df['videoId'].apply(lambda video_id: has_relevant_terms(video_id, relevant_terms))

filtered_df = df[(df['is_philippine'] == False) & (df['has_relevant_terms'] == True)]

# Save the filtered DataFrame to a new CSV file
output_csv = f'output_{csv_file}'  # Replace with the desired output file path
filtered_df.to_csv(output_csv, index=False)

print(f"Filtered channels and videos saved to {output_csv}")